In [35]:
%pip install fastapi SQLAlchemy sqlite3 aiosqlite databases

In [36]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.orm import sessionmaker

In [37]:
engine = create_engine("sqlite:///:memory:", echo=False)
metadata = MetaData()

users_table = Table(
    "users",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

metadata.create_all(engine)
SessionLocal = sessionmaker(bind=engine)

In [38]:
app = FastAPI()

class User(BaseModel):
    name: str

@app.get("/users")
async def get_users():
    session = SessionLocal()
    rows = session.query(users_table).all()
    session.close()
    return [{"id": r.id, "name": r.name} for r in rows]

@app.post("/users")
async def add_user(user: User):
    session = SessionLocal()
    row = users_table.insert().values(name=user.name)
    result = session.execute(row)
    session.commit()
    session.close()
    return {"id": result.lastrowid, "name": user.name}

In [39]:
import json

async def call_asgi(app, path, method="GET", body=None):
    response = {"status": None, "headers": None, "body": None}

    scope = {
        "type": "http",
        "method": method,
        "path": path,
        "raw_path": path.encode(),
        "query_string": b"",
        "headers": [],
    }

    messages = []

    async def receive():
        return {
            "type": "http.request",
            "body": body or b"",
            "more_body": False
        }

    async def send(message):
        if message["type"] == "http.response.start":
            response["status"] = message["status"]
            response["headers"] = message["headers"]
        elif message["type"] == "http.response.body":
            response["body"] = message.get("body", b"")

    await app(scope, receive, send)
    # decode body if JSON
    try:
        response["body"] = json.loads(response["body"].decode())
    except Exception:
        pass
    return response


In [40]:
# GET users (should be empty)
resp = await call_asgi(app, "/users")
print(resp)

# POST a new user
resp = await call_asgi(app, "/users", method="POST", body=b'{"name":"Alice"}')
print(resp)

# GET users again
resp = await call_asgi(app, "/users")
print(resp)

{'status': 200, 'headers': [(b'content-length', b'2'), (b'content-type', b'application/json')], 'body': []}
{'status': 200, 'headers': [(b'content-length', b'23'), (b'content-type', b'application/json')], 'body': {'id': 1, 'name': 'Alice'}}
{'status': 200, 'headers': [(b'content-length', b'25'), (b'content-type', b'application/json')], 'body': [{'id': 1, 'name': 'Alice'}]}
